#### «Скрытие данных в тексте методом изменения интервала между словами»

In [ ]:
#преобразует текст в двочный вид
def text2bit (TEXT):
  BYTE=TEXT.encode('utf-8')
  HEX=BYTE.hex()
  BIT=bin(int(HEX, base=16))[2:]
  return BIT

# вставляет пробел в строку по индексу
def insert_space(string, N):
    return string[:N] + ' ' + string[N:]

# считает сколько пробелов в строке
def check_space(string):
    count = 0
    for i in range(0, len(string)):
        if string[i] == " ": count += 1
    return count

# встраивание ЦВЗ в контейнер
def embedding(container, information):
    with open(information) as f:  # открытие файла
      msg=f.readline()  #чтение всего сообщения
    arr_bit= list(text2bit(msg)+'1000110010110100100011') #получение биты текста и добавление последовательности бит для обозначения окончания сообщения

    with open(container) as f:  # открытие файла
      lines=f.readlines()  #чтение всех строк файла в массив

    num_bit=len(arr_bit)  #определние сколько бит содержит текст


    index_bit=0
    text=[]
    for line in lines:  #проход по каждой строчке текста

      # тут добавляються пробелы между слов
      index_cursor=0
      for _ in range(check_space(line)):  #проход по каждому пробелу в строчке
          if index_bit < num_bit:  #проверка наличие бит для встраивания
            index_cursor=line.find(' ', index_cursor)  #поиск местоположения пробела в строке
            if arr_bit[index_bit] == '1':  #есди бит равен 1, то
                line=insert_space(line, index_cursor) #добавление второго пробела
                index_cursor+=2 #делаем смещение курсора (смещение на 2 из-за того, что 2 пробела)
            else: index_cursor+=1 #иначе просто делаем симщение курсора
            index_bit+=1 #увиличивание считчика для встраивание следующего бита

      text.append(line)#добавление обработанной строки
    with open('Стегфайл.txt', 'w') as f: #открытие файла для записи
      f.write(''.join(text))#записывание всех обработанных строк


# ==================================================================================================================================

# преобразует двочный вид в текст
def bit2text (BIT):
  BIT=BIT[:BIT.find('1000110010110100100011')]#нахождение последовательности бит которая обозначает окончания файла и обрезаем все после
  HEX=hex(int(BIT, 2))[2:]
  BYTE=bytes.fromhex(HEX)
  TEXT=BYTE.decode('utf-8')
  return TEXT

def counting_spaces(string):#подсчет сколько пробелов идет подрят
    result = []
    tmp = 0
    for i in string:
        if i == ' ':
            tmp += 1
        elif tmp != 0:
            result.append(tmp)
            tmp = 0
    return result

def extraction(steg_file):
    with open(steg_file) as f: #открытие файла
      text=f.readlines()#чтение всего файла

    arr_space=counting_spaces(''.join(text)) #получаем массив того сколько пробелов идет подрят: "Несколько    лет тому назад  \n" -> [4, 1, 1, 2]

    for i in range(len(arr_space)):
        arr_space[i] = 1 if arr_space[i] == 2 else 0 #(троичный оператор) если 2 пробела то заменяем на 1 иначе на 0

    bit= ''.join(map(str, arr_space)) #собирает все биты в идино
    text = bit2text(bit)#преобразует биты в текст
    with open('out_Сообщение.txt','w') as f:  f.write(text)

# ==================================================================================================================================

path = '/content/drive/MyDrive'
embedding(path+'Контейнер.txt', path+'int_Сообщение.txt')
extraction('Стегфайл.txt')

#### «Скрытие данных в пространственной области изображений методом замены наименее значащего бита»

In [ ]:
# from IPython.display import display
# display(Image.open(container))

from PIL import Image,ImageDraw
path = '/content/drive/MyDrive'

#преобразует текст в двочный вид
def text2bit (TEXT):
  BYTE=TEXT.encode('utf-8')
  HEX=BYTE.hex()
  BIT=bin(int(HEX, base=16))[2:]
  return BIT

def RGB(filename):
  img = Image.open(filename)
  img.show()
  return ([list(i)  for i in list(img.getdata())], img.size)

def NZB (num): return bin(num)[-1] #наименьший значащий бит(НЗБ)

def embeddingBIT (num, bit):
  bits = bin(num)[2:]
  bits = bits[:-1] + bit
  return int(bits, 2)

def saveImg(rgb, size):
  image = Image.new('RGB', size)
  image.putdata([tuple(i)  for i in rgb])
  image.save('Стегфайл.bmp')

def embedding(container, information):
  with open(information) as f: msg=f.readline()  #чтение всего сообщения
  arr_bit= text2bit(msg)+'1000110010110100100011' #получение биты текста и добавление последовательности бит для обозначения окончания сообщения
  rgb, size = RGB(container)

  for i, bit in enumerate(arr_bit):
    if bit != NZB(rgb[i][2]): #проверка совпадения НЗБ пикселя и встаиваемого бита информации
      if bit == '1': rgb[i][2] = embeddingBIT(rgb[i][2], '1')
      else: rgb[i][2] = embeddingBIT(rgb[i][2], '0')

  saveImg(rgb, size)

embedding(path+'Контейнер.bmp', path+'int_Сообщение.txt')
# ==================================================================================================================================

def bit2text (BIT):
  BIT=BIT[:BIT.find('1000110010110100100011')]#нахождение последовательности бит которая обозначает окончания файла и обрезаем все после
  HEX=hex(int(BIT, 2))[2:]
  BYTE=bytes.fromhex(HEX)
  TEXT=BYTE.decode('utf-8')
  return TEXT

def extraction(stegfile):
  rgb, _ = RGB(stegfile)
  bits = ''.join([NZB(i[2]) for i in rgb])
  text=bit2text(bits)
  with open('out_Сообщение.txt','w') as f:  f.write(text)

extraction('Стегфайл.bmp')

#### «Скрытие данных в пространственной области изображений методом псевдослучайного интервала»

In [ ]:
import random
from PIL import Image,ImageDraw
random.seed(30)
path = '/content/drive/MyDrive'

def text2bit (TEXT):
  BYTE=TEXT.encode('utf-8')
  HEX=BYTE.hex()
  BIT=bin(int(HEX, base=16))[2:]
  return BIT

def RGB(filename):
  img = Image.open(filename)
  img.show()
  return ([list(i)  for i in list(img.getdata())], img.size)

def embedding(container, information):
  with open(information) as f: msg=f.readline()  #чтение всего сообщения
  arr_bit= text2bit(msg)+'1000110010110100100011' #получение биты текста и добавление последовательности бит для обозначения окончания сообщения
  rgb, size = RGB(container)
  key = random.shuffle(list(range(size[0]*size[1])))
  print(key)
  key = key[:len(arr_bit)]
  print(key)
  for i, bit in enumerate(arr_bit):
    if bit != NZB(rgb[i][2]): #проверка совпадения НЗБ пикселя и встаиваемого бита информации
      if bit == '1': rgb[i][2] = embeddingBIT(rgb[i][2], '1')
      else: rgb[i][2] = embeddingBIT(rgb[i][2], '0')

  saveImg(rgb, size)

embedding(path+'Контейнер.bmp', path+'int_Сообщение.txt')